In [1]:
import os
import json
from utils import json_load, json_dump, mkdir

In [3]:
SOURCE_DIR = os.path.join('data', 'source')
data_file_path = os.path.join(SOURCE_DIR, 'gemma_response_with_proxy.json')
proxy_data = json_load(data_file_path)
proxy_data[0]

load data from: data/source/gemma_response_with_proxy.json


{'query': '題目: 常見針灸配穴法中,所指的「四關穴」,為下列何穴位之組合?\n選項:\n A: 上星、日月\n B: 合谷、太衝\n C: 內關、外關\n D: 上關、下關\n',
 'reference_answer': 'B',
 'response': {'ans': 'C',
  'feedback': '題目問的是「四關穴」，而參考資料中沒有提及四關穴的定義或組成。因此，無法根據提供的資料判斷正確答案。'},
 'wiki_results': {'四關穴': None,
  '合谷': {'title': '合谷穴',
   'url': 'https://zh.wikipedia.org/wiki/%E5%90%88%E8%B0%B7%E7%A9%B4',
   'summary': '合谷穴（LI 4）是手陽明大腸經的原穴，出自《靈樞·本輸》，又名虎口。“合”意即合攏，“谷”是山谷的意思。此穴在第一、二掌骨之間，兩骨相合，形狀如山谷的地方，所以名為合谷。又因位於手拇指虎口兩骨之間，所以又稱為虎口。',
   'content': '合谷穴（LI 4）是手陽明大腸經的原穴，出自《靈樞·本輸》，又名虎口。“合”意即合攏，“谷”是山谷的意思。此穴在第一、二掌骨之間，兩骨相合，形狀如山谷的地方，所以名為合谷。又因位於手拇指虎口兩骨之間，所以又稱為虎口。\n\n\n== 定位 ==\n位於手背，第一、二掌骨之間，約當第二掌骨橈側的中點處取穴。\n\n\n== 取穴法 ==\n簡便取穴法：以一手的拇指指骨關節橫紋，放在另一手拇、食指之間的指蹼緣上，當拇指尖下是穴。\n\n《針灸甲乙經》：在手大指次指歧骨間。\n《針灸大成》：在手大指次指歧骨間陷中。\n\n\n== 局部解剖 ==\n在第一、二掌骨之間，第一骨間背側肌中，深層有拇內收肌橫頭。有手背靜脈網。腧穴近側正當橈動脈從手背穿向手掌之處。布有橈神經淺支的掌背側神經，深部有正中神經的指掌側固有神經。\n\n\n== 刺灸法 ==\n直刺0.5-0.8寸，可灸。\n\n《針灸大成》：針三分, 留六呼, 灸三壯。合谷，婦人妊娠可瀉不可補，補即墮胎。\n\n\n== 主治 ==\n功能：清熱解表，明目聰耳，鎮靜止痛\n主治：\n\n本經脈所過部位的疾患：指孿，臂痛，半身不遂。\n外感疾患：熱病無汗，多汗，咳

In [4]:
import os
from dotenv import find_dotenv, load_dotenv
_ = load_dotenv(find_dotenv())

from llama_index.llms.openai import OpenAI
from llama_index.llms.ollama import Ollama

llm = OpenAI(model="gpt-5-mini", temperature=0, is_streaming=False, response_format={"type": "json_object"})

In [5]:
DEST_DIR = os.path.join('data', 'temp')
mkdir(DEST_DIR)

Directory 'data/temp' created successfully.


In [9]:
from llama_index.core.prompts import PromptTemplate

RETRIEVAL_EVALUATOR_PROMPT = PromptTemplate(
    template="""你是一個具備判斷 retrieval 資料完整性與可靠性的助手。

輸入：
- query（string）：一題單選題（例如 題目: 常見針灸配穴法中,所指的「四關穴」,為下列何穴位之組合?\n選項:\n A: 上星、日月\n B: 合谷、太衝\n C: 內關、外關\n D: 上關、下關\n）
- context（string）：經過 Wiki 或其它來源檢索到的 text 資料，用來支持回答這個 query

你的任務是：
1. 判斷這個 context 是否已經足夠用來「正確地回答」這個單選題(correct)，或是不行(incorrect)、或是有模糊性(ambiguous)。
2. 輸出一個 JSON 物件，格式如下：

{
  "verdict": "<correct 或 incorrect 或 ambiguous>",
  "next_query": "<可選的重寫 query，用於進一步檢索；當判斷是 correct 時可為空字串或 null>"
}

* 若判斷為 **correct**，表示 context 足夠回答問題，`next_query` 可以設為 `""` 或 `null`。
* 若判斷為 **incorrect**，表示 context 與題意不匹配、或根本無法支撐任何選項，`next_query` 要提供一個你幫我重寫（或擴增）的 query，我會拿它去做新的檢索。
* 若判斷為 **ambiguous**，表示 context 有部分相關性但仍有不確定性，或答案可能不唯一，這時 `next_query` 也是必須的：你提供一個 query，用來補充搜索。未來當我拿新的檢索結果補充時，我會把原 context 與新的結果 concat 在一起，再讓你重做這個判斷流程。

**注意事項 / 提示給 LLM：**

* `判斷標籤` 的三種情況應該代表不同策略：
    • **correct**：context 已經明確支持某個選項，沒有爭議或缺漏
    • **incorrect**：context 與題目方向錯誤（例如回答了另一個問題、講了題目沒有問的範圍）
    • **ambiguous**：context 涉及題目相關內容，但缺少關鍵細節或多種可能性尚未排除
* 在決定 `next_query` 時，盡量把 query 重寫得更具體／更具針對性（可加入關鍵詞）以便後續檢索能補足缺口
* 輸出 JSON 時，**只有這兩個欄位**，不要額外說明或多餘包裝文字

---

### 範例（示意）

假設


query = "哪一年諾貝爾獎創立？"
context = "諾貝爾獎是瑞典的獎項，始於十九世紀末…在 1901 年頒發第一屆。"


那麼 LLM 可以回：

{
  "verdict": "correct",
  "next_query": null
}

又假設


query = "哪一年諾貝爾獎創立？"
context = "諾貝爾獎由阿爾弗雷德·諾貝爾設立"


那麼就可能回：


{
  "verdict": "ambiguous",
  "next_query": "諾貝爾獎 第一屆 頒發 年份"
}


或若 context 完全與題目無關，標記為 incorrect，並重寫一個可能的 query。
----------以下開始

query = {query}
context = {context}"""
)

In [11]:
import json

In [12]:
rvs = []
for idx, data in enumerate(proxy_data):
    print(idx, end=': ')
    rv = data.copy()
    query = data['query']
    context = data['context']
    prompt = RETRIEVAL_EVALUATOR_PROMPT.format(query=query, context=context)
    response = llm.complete(prompt)
    rv['evaluator_result'] = json.loads(response.text)
    print(json.loads(response.text)['verdict'])
    rvs.append(rv)

0: incorrect
1: correct
2: correct
3: incorrect
4: ambiguous
5: incorrect
6: ambiguous
7: correct
8: incorrect
9: ambiguous


In [13]:
save_file_path = os.path.join(DEST_DIR, 'retrieval_evaluator.json')
json_dump(save_file_path, rvs)

dump result to: data/temp/retrieval_evaluator.json


In [14]:
for rv in rvs:
    er = rv['evaluator_result']
    print(er)

{'verdict': 'incorrect', 'next_query': '四關穴是哪些穴位組合？(查詢：四關穴 合谷 太衝 內關 外關 上星 日月 上關 下關)'}
{'verdict': 'correct', 'next_query': None}
{'verdict': 'correct', 'next_query': None}
{'verdict': 'incorrect', 'next_query': '外丘 穴位 國際譯名 國際編號 WHO 外丘 (Waiqiu) 編號 GB33 GB34 GB35 GB36'}
{'verdict': 'ambiguous', 'next_query': '陰維脈 郄穴 是 哪一個？（交信、築賓、府舍、腹哀）'}
{'verdict': 'incorrect', 'next_query': '足太陰脾經 络穴 哪一穴？地機 漏谷 公孫 太白'}
{'verdict': 'ambiguous', 'next_query': '《難經·六十八難》 心下滿 主何穴？ 前谷 液門 湧泉 跗陽'}
{'verdict': 'correct', 'next_query': None}
{'verdict': 'incorrect', 'next_query': '臑會、顴髎、秉風、聽宮、耳門 哪些屬於三焦經與膽經的交會穴'}
{'verdict': 'ambiguous', 'next_query': '通里穴 (HT5) 是否屬於 馬丹陽 天星十二穴？請列出馬丹陽天星十二穴的完整清單或來源。'}


In [15]:
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

In [16]:
from llama_index.tools.tavily_research.base import TavilyToolSpec
tavily_tool = TavilyToolSpec(
    api_key=TAVILY_API_KEY,
)

In [19]:
test_response = tavily_tool.search('search test', max_results=1)
test_response

[Document(id_='c52e1178-a1c6-4d0d-83a1-10bb28c9e6b9', embedding=None, metadata={'url': 'https://en.wikipedia.org/wiki/Wikipedia:Search_engine_test'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='Depending on the subject matter, and how carefully it is used, a search engine test can be very effective and helpful, or produce misleading or non-useful results. In most cases, a search engine test is a first-pass heuristic or "rule of thumb".\n\n### What a search test can do, and what it can\'t\n\nA search engine can index pages and text which others have placed on the internet, just like a big index at the back of a book.\n\nSearch engines can: [...] | Finding vaguely remembered information and unfamiliar terms. | [...] 1 Some search engine tests\n 2 Good-faith searching: a rule of thumb\n 3 Search engine tests\n  + 3.1 What a search

In [23]:
for doc in test_response:
    print(doc.metadata['url'])
    print(doc.text_resource.text)

https://en.wikipedia.org/wiki/Wikipedia:Search_engine_test
Depending on the subject matter, and how carefully it is used, a search engine test can be very effective and helpful, or produce misleading or non-useful results. In most cases, a search engine test is a first-pass heuristic or "rule of thumb".

### What a search test can do, and what it can't

A search engine can index pages and text which others have placed on the internet, just like a big index at the back of a book.

Search engines can: [...] | Finding vaguely remembered information and unfamiliar terms. | [...] 1 Some search engine tests
 2 Good-faith searching: a rule of thumb
 3 Search engine tests
  + 3.1 What a search test can do, and what it can't
 4 Search engine tests and Wikipedia policies
  + 4.1 Verifiability
  + 4.2 Neutrality
  + 4.3 Notability
 5 Using search engines
  + 5.1 Search engine expressions (examples and tutorial))
  + 5.2 Specific uses of search engines in Wikipedia
 6 Interpreting results
  + 6.1 

In [26]:
data['context']

'-----搜索關鍵字: 通里穴\n條目名稱: 通里穴\n內容: 通里穴(Tongli,HT5)，穴道名，屬手少陰心經絡穴。通，通往，里，內裏。心經絡脈由此穴分出，與小腸經互為表裡而相通。\n\n\n== 定位 ==\n腕橫紋上1寸，尺側腕曲肌腱的橈側。\n解剖：在尺側腕曲肌與指淺曲肌之間，深層為指伸曲肌，有尺動脈通過。布有前臂內側皮神經，尺側為尺神經。\n\n\n== 刺灸法 ==\n直刺0.3─0.5寸\n針0.3─0.5寸，灸2─3壯\n\n\n== 主治 ==\n心悸、怔忡、暴喑、舌強不語、腕臂內側痛、肘及前臂疼痛、頭痛、目眩、面赤熱、遺尿、月經過多、狂症\n\n\n== 配伍 ==\n係心經絡穴，別走手太陽小腸經，有鎮靜、安神、調心氣的作用。配內關、心俞治心絞痛、心律不齊。\n\n\n== 參考資料 ==\n\n\n== 參看 ==\n腧穴列表\n----------搜索關鍵字: 靈道穴\n條目名稱: 靈道穴\n內容: 靈道穴(Lingdao, HT4)，穴道名，屬手少陰心經，亦為五輸穴的經穴，五行屬金。靈，神靈；道，通道。心主神靈，此穴在尺側腕曲肌腱橈側溝，猶如通向神靈之道。\n\n\n== 定位 ==\n腕橫紋上1.5寸，尺側腕曲肌腱的橈側。\n解剖：在尺側腕曲肌與指淺曲肌之間，深層為指伸曲肌，有尺動脈通過。布有前臂內側皮神經，尺側為尺神經。\n\n\n== 刺灸法 ==\n直刺0.3─0.5寸\n針0.2─0.4寸，灸2─3壯\n\n\n== 主治 ==\n心痛、胃脘疼痛、乾嘔、瘈瘲、暴喑、肘臂攣痛、臂外廉疼痛、目赤腫、手濕癢不仁、癔病\n\n\n== 參考資料 ==\n\n\n== 參照 ==\n腧穴列表\n-----'

In [27]:
def get_tavily_context(tavily_response):
    txt = ''
    rvs = []
    for doc in tavily_response:
        txt+=doc.text_resource.text
        txt+='\n\n'
        rvs.append({
            'url': doc.metadata['url'],
            'txt': doc.text_resource.text
        })
    return txt, rvs

In [17]:
tavily_source = rvs.copy()

In [30]:
next_query

'incorrect'

In [32]:
rvs = []
for data in tavily_source:
    rv = data.copy()
    verdict = rv['evaluator_result']['verdict']
    if verdict in ['incorrect', 'ambiguous']:
        next_query = rv['evaluator_result']['next_query']
        tavily_response = tavily_tool.search(next_query, max_results=5)
        context, response = get_tavily_context(tavily_response)
        rv['tavily_response'] = response
    else:
        context = ''
        rv['tavily_response'] = None

    if verdict == 'incorrect':
        old_context = ''
    else:
        old_context = rv['context']

    context_v2 = context + old_context
    print(context_v2)
    rv['context_v2'] = context_v2
    rvs.append(rv)

合谷、太冲两对穴位一上一下，上下对应，好似把住人体的主要关口。 · 太冲穴属足厥阴肝经之原穴；合谷穴属手阳明大肠经之原穴。 · 太冲穴属足厥阴肝经，为少气多

四关穴是合谷穴与太冲穴的合称，左右各一对，合称四关。四关穴具有疏肝理气、调和气血、通经活络等功效，在中医临床中常用于调理气机不畅、经络不通等证

四关穴与气血、阴阳学说也有着密切的关系。合谷为大肠原穴，为阳，主气；太冲，肝经原穴，为阴，主血，两者可谓气化功能之关键，气血通行之要塞，以应“气行则血行，

如太冲、阳陵泉等则从整体上调整机体的气血和经络，体现了中医的整体观念；组合②有清解热邪、祛 瘀通络、舒筋止痛的作用，对于辩证为血热证的PHN 疗效佳；组合③支沟穴《类经图翼》中指出：“凡 三焦相火炽盛，及大便不通，胁肋疼痛者，俱得泻之。”即支沟穴具疏利三焦，缓解疼痛的作用，亦可辩 证搭配组合②加强泄热的作用；组合④为足太阴脾经上的穴位，脾喜燥恶湿，为后天之本，所有的生命 活动都有赖于后天的脾胃摄入营养物质提供能量，脾的功能状态直接影响到人体的健康和疾病状态，故 取阴陵泉祛湿健脾和胃之效。组合⑤有助于调节机体的免疫功能，改善局部血液循环，减轻炎症反应， 从而缓解PHN 的症状。在TN 的聚类分析中同样得到5 个核心聚类群，组合①：合谷–下关–四白–地 仓–内庭–太冲；组合②：颧髎–迎香；组合③：颊车–翳风；组合④：攒竹–阳白；组合⑤：风池–太 阳。组合①为治疗TN 的常用穴位，体现了针灸中“远近配伍”的配穴原则，还有“开四关”的治疗方 法，《标幽赋》中提出：“拘挛闭塞，遣八邪而去矣；寒热痹痛，开四关而已之。”即合谷、太冲配伍可 [...] 下关(79)风池(60)四白(53)地仓(41)攒竹(40)颊车(33)太阳(30)颧髎(28)阳白 (25)迎香(24)翳风(21)承浆(18)天柱(18)神庭(17)完骨(17)印堂(17)鱼腰(16) 水沟(15)丝竹空(12)百会(11)头维(10)夹承浆(10)大迎(5)巨髎(5)风府(4)上 关(4)颅息(3)听宫(3)列缺(2)大椎(1)耳门(1)颔厌(1)上星(1)四神聪(1) 麦书铭 等 DOI: 10.12677/tcm.2025.142118 790 中医学 续表 胸腹部 3 (0.33%) 3 (5.45%) 期门(1)气海(1)膻中(1) 腰背部 4 

In [34]:
save_file_path = os.path.join(DEST_DIR, 'tavily_result.json')
json_dump(save_file_path, rvs)

dump result to: data/temp/tavily_result.json


In [35]:
ANSWER_PROMPT_WITH_CONTEXT = PromptTemplate(
    template="""
你是一個中醫考題專家，請根據下面的題目回答單選題。

請遵守以下規則：
1. 嚴格依據提供的參考資料 context 作答。
2. 輸出 JSON 格式。
3. JSON 需包含兩個 key：
   - "ans" ：只回答單選答案 (A/B/C/D)
   - "feedback" ：簡短說明為什麼選這個答案
4. 不要加入題目之外的說明或其他文字。

題目：
#{query}

參考資料 (context)：
#{context}

請直接輸出 JSON：
"""
)


In [36]:
tavily_results = rvs.copy()
llm = Ollama(model="gemma3:12b", temperature=0.0, request_timeout=120.0, context_window=8000, json_mode=True)
DEST_DIR = os.path.join('data', 'deliverables')
mkdir(DEST_DIR)
save_file_path = os.path.join(DEST_DIR, 'gemma_response_with_tavily.json')


Directory 'data/deliverables' created successfully.


In [38]:
rvs = []
for idx, tavily_result in enumerate(tavily_results):
    print(f"idx: {idx}", end=', ')
    rv = tavily_result.copy()

    query = tavily_result['query']
    context = tavily_result['context_v2']
    prompt = ANSWER_PROMPT_WITH_CONTEXT.format(query=query, context=context)
    #print(prompt)
    response = llm.complete(prompt)
    rv['response_v2'] = json.loads(response.text)
    rvs.append(rv)

json_dump(save_file_path, rvs)

idx: 0, idx: 1, idx: 2, idx: 3, idx: 4, idx: 5, idx: 6, idx: 7, idx: 8, idx: 9, dump result to: data/deliverables/gemma_response_with_tavily.json


In [39]:
correct = 0
incorrect = 0
for rv in rvs:
    reference_answer = rv['reference_answer']
    pred_ans = rv['response_v2']['ans']
    if reference_answer == pred_ans:
        correct+=1
    else:
        incorrect+=1

print(f"{correct}/{correct + incorrect}")

7/10
